In [1]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''

  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [2]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]
!pip install PyMuPDF
import fitz  # PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 84.6 MB/s eta 0:00:00


In [3]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [4]:
# # open and read the file from google drive
# file = open("/content/drive/MyDrive/Colab Notebooks/Data_Pricing_in_Machine_Learning_Pipelines.txt", "r")
# FileContent = file.read().strip()
# Path to the PDF file
pdf_path = "/content/A_Unified_Analytical_Framework_for_Trustable_Machine_Learning_and__Automation_Running_with_Blockchain.pdf"

# Extract text from the PDF file
FileContent = extract_text_from_pdf(pdf_path)

In [5]:
FileContent

'© 2018 IEEE. Personal use of this material is permitted. Permission from IEEE must be obtained for all other uses, in any current or future media, including \nreprinting/republishing this material for advertising or promotional purposes, creating new collective works, for resale or redistribution to servers or lists, or reuse \nof any copyrighted component of this work in other works. \nA Unified Analytical Framework for Trustable Machine \nLearning and Automation Running with Blockchain \nTao Wang  \nSAS Institute Inc. \nCary, USA \nt.wang@sas.com \nAbstract—Traditional machine learning algorithms use data \nfrom databases that are mutable, and therefore the data cannot \nbe fully trusted. Also, the machine learning process is difficult to \nautomate. This paper proposes building a trustable machine \nlearning system by using blockchain technology, which can store \ndata in a permanent and immutable way. In addition, smart \ncontracts are used to automate the machine learning process

In [6]:
len(FileContent)

45954

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [8]:
tokenizer.model_max_length

1024

In [9]:
tokenizer.max_len_single_sentence

1022

In [10]:
tokenizer.num_special_tokens_to_add()

2

In [11]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

Token indices sequence length is longer than the specified maximum sequence length for this model (1612 > 1024). Running this sequence through the model will result in indexing errors


1612

In [13]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

11

In [14]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1020, 977, 997, 988, 1006, 1016, 994, 1012, 959, 764, 641]

In [15]:
[len(tokenizer(c).input_ids) for c in chunks]

[1022, 979, 999, 990, 1008, 1018, 996, 1014, 961, 766, 643]

In [16]:
sum([len(tokenizer(c).input_ids) for c in chunks])

10396

In [17]:
len(tokenizer(FileContent).input_ids)

12348

In [18]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

10374

In [19]:
len(tokenizer.tokenize(FileContent))

12346

In [20]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  summary_generated = tokenizer.decode(*output, skip_special_tokens=True)
  print(summary_generated)

 Traditional machine learning algorithms use data from databases that are mutable, therefore the data cannot be fully trusted. This paper proposes building a trustable machine. learning system by using blockchain technology, which can store. data in a permanent and immutable way. In addition, smart.contracts are used to automate the machine learning process.
 A paper proposes to use the smart contract as the automation engine to solve the automation problem. It establishes a link between machine learning technology and blockchain technology. The proposed unified analytical framework solves both the trustability and automation issues in machine learning. It also makes it possible for a computer to translate core machine learning implementations from a single thread on a single machine to multiple machines running with blockchain.
 This paper uses these seven steps to demonstrate the framework for trustable machine learning with blockchain. Machine learning builds probabilistic models ba

In [24]:
len(summary_generated)

260